In [1]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters

In [2]:
def SX2DM(sx):
    return cs.DM(sx)

In [3]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['l_hip_3', 'l_lower_leg']
# Specify the root link
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link)

Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor


In [4]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =np.random.rand(len(link_name_list))  
lenght_multiplier = np.random.rand(len(link_name_list))
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

In [5]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()
print("M", M(H_b, s, density, lenght_multiplier))
print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

l_hip_3
l_lower_leg
l_hip_3
l_lower_leg
l_hip_3
l_lower_leg
M @1=4.44089e-16, @2=5.15997, @3=5.67605, @4=-3.66832, @5=0.14574, @6=0.0700873, @7=-0.448417, @8=-0.00290462, @9=0.0554759, @10=0.245727, @11=0.121016, @12=-0.0183146, @13=0.0731262, @14=0.205368, @15=0.574548, @16=-0.00633713, @17=0.255028, @18=0.000489173, @19=0.00112887, @20=1.01875, @21=4.04523, @22=0.043595, @23=2.21226, @24=-0.0919223, @25=-0.00303776, @26=2.78616, @27=8.25917, @28=0.807359, @29=-0.0327862, @30=0.103722, @31=0.00103376, @32=-0.00495617, @33=0.194324, @34=-0.857983, @35=-0.0576868, @36=-0.0566723, @37=-0.193132, @38=-0.0539441, @39=-0.101013, @40=-0.0807005, @41=0.0688655, @42=-0.0134997, @43=0.632221, @44=-4.45633, @45=-1.02993, @46=-0.0522624, @47=0.0223326, @48=0.0139563, @49=-2.63374, @50=1.54066, @51=-0.0992179, @52=0.0321824, @53=0.411859, @54=-0.000320923, @55=0.0476884, @56=0.815368, @57=0.169636, @58=-0.0571701, @59=0.039014, @60=-0.157066, @61=-0.327966, @62=0.0575446, @63=-0.0405341, @64=0.102

In [6]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

65.9308


In [7]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

@1=0, 
[[-0.518298, 0.0753914, -0.85187, 0.141924], 
 [-0.400406, -0.901574, 0.163826, -1.06045], 
 [-0.755674, 0.426004, 0.497471, 1.98366], 
 [@1, @1, @1, 1]]


In [8]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

[0.168882, -1.06416, 2.12015]


In [9]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

@1=1, @2=0, 
[[@1, @2, @2, @2, -0.0665942, 0.0549292, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.00115904, 0.00414005, -0.00701059, 0.0211125, 0.0699944, 0.00496075, @2, @2, @2, @2, @2, @2], 
 [@2, @1, @2, 0.0665942, @2, 0.0290823, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.0325323, -0.0181252, 0.0217498, -0.0339975, 0.0297776, -0.0593236, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @1, -0.0549292, -0.0290823, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.0185769, -0.0444482, -0.00646224, -0.0439159, 0.0659764, 0.0280311, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.753265, -0.13395, 0.530995, 0.171116, -0.171116, -0.518298, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.34618, 0.913201, 0.394047, 0.817198, -0.817198, -0.400406, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.559242, -0.384865, 0.750181, -0.55037, 0.55037, -0.7556

In [10]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0262051, 0.0319903, 0.00983049, 0.0335666, -0.0123539, -0.0417079, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0280134, 0.00136546, 0.0204256, -0.0999295, 0.00112373, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0267972, -0.0226536, 0.0215745, -0.0445674, -0.000533377, -0.0508805, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.577912, -0.239619, 0.816777, -0.816777, 0.247582, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -0.955924, -0.104004, 0.104004, 0.951632, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.816099, 0.169684, 0.567501, -0.567501, -0.181931, 0, 0, 0, 0, 0, 0]]


In [11]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

[-42.462, 35.4132, 620.479, 36.5304, -31.068, -36.9931, -35.8595, 9.54733, -9.53651, 0.546941, 6.70123, -0.00679463, 0.606893, 9.11208, 1.45321, -1.15964, 1.12046, 1.86833, -3.46951, 0.163838, 3.80663, -3.16627, 0.783866, 3.41061, -47.5998, -13.4679, -31.7802, 13.3327, -9.17099]


In [12]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Bias force term:
 [-42.462, 35.4132, 620.479, 36.5304, -31.068, -36.9931, -35.8595, 9.54733, -9.53651, 0.546941, 6.70123, -0.00679463, 0.606893, 9.11208, 1.45321, -1.15964, 1.12046, 1.86833, -3.46951, 0.163838, 3.80663, -3.16627, 0.783866, 3.41061, -47.5998, -13.4679, -31.7802, 13.3327, -9.17099]


In [13]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Coriolis force term:
 [-42.462, 35.4132, -26.0813, 71.2227, 11.0532, -36.9931, -10.0313, -5.56143, -8.56352, 0.231339, 2.66228, -0.00364745, 0.139229, 1.11606, -0.21035, -0.598991, 0.73786, 3.40863, -0.253272, -0.400482, 4.20413, -4.17052, 0.721312, 20.7051, -55.1405, -22.1652, -27.568, 12.8991, -9.21346]


In [14]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

Gravity term:
 [-7.10543e-14, -4.26326e-14, 646.56, -34.6923, -42.1212, 3.55271e-15, -25.8282, 15.1088, -0.972995, 0.315602, 4.03896, -0.00314718, 0.467664, 7.99602, 1.66356, -0.560647, 0.382596, -1.54029, -3.21624, 0.56432, -0.397504, 1.00425, 0.0625541, -17.2945, 7.54066, 8.69734, -4.21221, 0.433589, 0.0424717]
